# Portfolio Value at Risk (VaR) Calculator

This notebook demonstrates VaR calculation using three industry-standard methods:
1. **Historical VaR** - Percentile-based approach
2. **Parametric VaR** - Variance-covariance method
3. **Monte Carlo VaR** - Simulation-based

Author: Avni Derashree

In [ ]:
# Import libraries
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_loader import get_sample_portfolio, fetch_stock_data, calculate_returns, calculate_portfolio_returns
from src.var_models import calculate_all_var, var_summary_table, historical_var, parametric_var, monte_carlo_var
from src.backtesting import rolling_var_backtest, backtest_summary_table
from src.visualization import plot_return_distribution, plot_var_comparison, plot_portfolio_performance, plot_var_breaches

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

## 1. Load Portfolio Data

In [ ]:
# Fetch sample portfolio data
prices, portfolio_returns, tickers = get_sample_portfolio(period='2y')

print(f"Portfolio: {tickers}")
print(f"Date Range: {prices.index[0].strftime('%Y-%m-%d')} to {prices.index[-1].strftime('%Y-%m-%d')}")
print(f"Trading Days: {len(prices)}")

In [ ]:
# Portfolio statistics
stats = pd.DataFrame({
    'Metric': ['Mean Daily Return', 'Daily Volatility', 'Annualized Volatility', 
               'Skewness', 'Kurtosis', 'Min Return', 'Max Return'],
    'Value': [
        f"{portfolio_returns.mean():.4%}",
        f"{portfolio_returns.std():.4%}",
        f"{portfolio_returns.std() * np.sqrt(252):.2%}",
        f"{portfolio_returns.skew():.3f}",
        f"{portfolio_returns.kurtosis():.3f}",
        f"{portfolio_returns.min():.4%}",
        f"{portfolio_returns.max():.4%}"
    ]
})
stats

In [ ]:
# Visualize portfolio performance
fig = plot_portfolio_performance(prices, portfolio_returns, figsize=(14, 10))
plt.show()

## 2. Calculate Value at Risk

### VaR Definition
VaR answers: *"What is the maximum loss I can expect over a given time period, at a specific confidence level?"*

For example, a 1-day 95% VaR of -1.65% means:
- With 95% confidence, the portfolio will not lose more than 1.65% in a single day
- Or equivalently, there's a 5% chance of losing more than 1.65%

In [ ]:
# Calculate VaR using all three methods
var_results = calculate_all_var(portfolio_returns, n_simulations=10000)

# Display summary table
var_summary_table(var_results)

In [ ]:
# Dollar VaR for a $1M portfolio
portfolio_value = 1_000_000

print(f"Daily VaR for ${portfolio_value:,.0f} Portfolio:")
print("-" * 50)
for method, result in var_results.items():
    var_95_dollar = abs(result.var_95) * portfolio_value
    var_99_dollar = abs(result.var_99) * portfolio_value
    print(f"{result.method:<15} 95%: ${var_95_dollar:>10,.0f}   99%: ${var_99_dollar:>10,.0f}")

In [ ]:
# Visualize VaR on return distribution
fig = plot_return_distribution(portfolio_returns, var_results, confidence_level=0.95, figsize=(12, 6))
plt.show()

In [ ]:
# Compare VaR and Expected Shortfall across methods
fig = plot_var_comparison(var_results, figsize=(10, 6))
plt.show()

## 3. Backtest VaR Models

Backtesting validates whether the VaR model is accurate by:
1. Computing rolling VaR estimates using historical data
2. Counting how often actual losses exceed VaR ("breaches")
3. Comparing breach rate to expected rate (e.g., 5% for 95% VaR)

In [ ]:
# Run backtests for all methods
backtest_results = []
var_series_dict = {}
breach_dict = {}

for var_func, name in [(historical_var, 'Historical'), 
                        (parametric_var, 'Parametric'), 
                        (monte_carlo_var, 'Monte Carlo')]:
    var_series, breaches, result = rolling_var_backtest(
        portfolio_returns, 
        var_func, 
        window=252, 
        confidence_level=0.95
    )
    backtest_results.append(result)
    var_series_dict[name] = var_series
    breach_dict[name] = breaches

# Display backtest summary
backtest_summary_table(backtest_results)

In [ ]:
# Visualize backtest breaches for Historical VaR
fig = plot_var_breaches(
    portfolio_returns,
    var_series_dict['Historical'],
    breach_dict['Historical'],
    method_name='Historical VaR (95%)',
    figsize=(14, 8)
)
plt.show()

## 4. Key Insights

Based on this analysis:

In [ ]:
print("🔍 KEY FINDINGS")
print("=" * 50)

# Kurtosis analysis
kurt = portfolio_returns.kurtosis()
print(f"\n1. Distribution Shape:")
print(f"   Kurtosis = {kurt:.2f}")
if kurt > 0:
    print("   → Fat tails present (leptokurtic)")
    print("   → Parametric VaR likely underestimates risk")

# Breach comparison
print(f"\n2. Backtest Performance (95% VaR):")
for r in backtest_results:
    status = "✓" if r.breach_rate <= 0.06 else "⚠️"
    print(f"   {status} {r.method}: {r.breach_rate:.1%} breaches (expected 5%)")

# Best method
best = min(backtest_results, key=lambda x: abs(x.breach_rate - 0.05))
print(f"\n3. Best Performing Method: {best.method}")
print(f"   Breach rate closest to expected 5%")

# Expected Shortfall comparison
print(f"\n4. Expected Shortfall (99%):")
for method, result in var_results.items():
    print(f"   {result.method}: {result.es_99:.2%}")

## 5. Conclusion

This analysis demonstrates:

1. **Historical VaR** performs well for fat-tailed distributions
2. **Parametric VaR** may underestimate tail risk when returns are non-normal
3. **Monte Carlo VaR** provides flexibility for complex portfolios
4. **Expected Shortfall (CVaR)** captures tail risk beyond VaR

For production risk management, consider:
- Using multiple VaR methods and taking the most conservative
- Regular backtesting to validate model accuracy
- Stress testing for extreme scenarios not captured in historical data